In [12]:
from numpy.fft import fft
import numpy as np
import random
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

In [13]:
N_data = 2500  # Number of training data in final augmented dataset
L = 1024      # Length of IQ sequence in each sample
D = 100 # Decimation factor for long-term data
SNRdBmin = 10
SNRdBmax = 20

max_pw  = 1000 # Pulse Width (us)
max_pri = 1000 # Pulse Repetition Interval (us)
SNRdBmin = 0
SNRdBmax = 20
fs = 20e6
LFM_maxBW = 10e6
LFM_minBW = 1e6
max_agility_BW = 5e6
min_agility_BW = 1e6
seq_len = 1000

checkpoint_filepath = 'temp/checkpoint/pri_pw05_cont.h5'
log_filepath = 'temp/tb_logs'

N_train_iter = 100

In [14]:
def gen_data():
    data = np.zeros((N_data, L, 8))
    labels = np.zeros((N_data, 4)) # 4 columns: 0 is pulse width, 1 is PRI, 2 is SNRdB, 3 is mod_type

    for i in range(N_data):
        # Select Radar emitter parameters
        pri_min = random.uniform(0, np.log10(max_pri)) 
        pri_min = 10**pri_min
        pw_max = random.uniform(0, np.log10(min(pri_min, max_pw))) # PW needs to be less than pri, or the radar will be CW
        pw_max = 10**pw_max
        phase_off = random.uniform(0, 2*np.pi)

        pulse_phase = random.uniform(0, 1) # Phase between 0 and 1 as percent of cycle elapsed at start of waveform
        mod_type = random.randint(0,1) # 0 is unmodulated, 1 is LFM
        freq_agile = random.randint(0,1) # 0 is no agility, 1 is jitter
        pw_agile = random.randint(0,1) # 0 is no agility, 1 is jitter
        pri_agile = random.randint(0,1) # 0 is no agility, 1 is jitter
        freq_agility_BW = random.uniform(min_agility_BW, max_agility_BW)
        LFM_BW = random.uniform(np.log10(LFM_maxBW), np.log10(LFM_minBW))
        LFM_BW = 10**LFM_BW

        freq_seq = np.zeros(seq_len)
        freq_off_seq = np.zeros(seq_len)
        if (freq_agile == 1):
            for j in range(seq_len):
                freq_seq[j] = random.uniform(-LFM_BW/2, LFM_BW/2) # deviation of frequency from center frequency
                freq_off_seq[j] = random.uniform(0, 2*np.pi) # phase offset that varies per pulse

        pw_seq = np.zeros(seq_len)
        if (pw_agile == 1):
            pw_min = random.uniform(np.log10(1), np.log10(pw_max))
            pw_min = 10**pw_min
            for j in range(seq_len):
                pw_seq[j] = random.uniform(pw_min, pw_max) 
        else:
            pw_min = pw_max

        pri_seq = np.zeros(seq_len)
        if (pri_agile == 1):
            pri_max = random.uniform(np.log10(pri_min), np.log10(max_pri))
            pri_max = 10**pri_max
            for j in range(seq_len):
                pri_seq[j] = random.uniform(pri_min, pri_max)
        else:
            pri_max = pri_min


        t = np.linspace(0.0, (L*D-1)/fs, L*D)
        dt = t[1]-t[0]

        x_mag = np.zeros(D*L)
        x_phase = np.zeros(D*L)
        # No modulation or agility
        if (mod_type == 0) & (pw_agile == 0) & (pri_agile == 0) & (freq_agile == 0):
            x_mag = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_min*1e-6)
            x_phase = phase_off
        # basic LFM
        elif (mod_type ==1) & (pw_agile == 0) & (pri_agile == 0) & (freq_agile == 0):
            x_mag   = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_min*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            x_phase = np.pi*LFM_BW*(x_time**2/(pw_min*1e-6)-x_time)
        # unmod freq_agile
        elif (mod_type ==0) & (freq_agile == 1) & (pw_agile == 0) & (pri_agile == 0):
            x_mag   = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_min*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int) # Turn the floats in the array into integers so that they can be used for indexing. Lord only knows why integer division returns a float
            x_phase = 2*np.pi*freq_seq[k % seq_len]*x_time+freq_off_seq[k % seq_len]
        # LFM freq_agile
        elif (mod_type ==1) & (freq_agile == 1) & (pw_agile == 0) & (pri_agile == 0):
            x_mag   = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_min*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int)
            x_phase = np.pi*LFM_BW*(x_time**2/(pw_min*1e-6)-x_time)+2*np.pi*freq_seq[k % seq_len]*x_time+freq_off_seq[k % seq_len]
        # unmod pw_agile
        elif (mod_type == 0) & (freq_agile == 0) & (pw_agile == 1) & (pri_agile == 0):
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int)
            x_mag = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_seq[k % seq_len]*1e-6)
            x_phase = phase_off
        # LFM pw_agile
        elif (mod_type == 1) & (freq_agile == 0) & (pw_agile == 1) & (pri_agile == 0):
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int)
            x_mag = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_seq[k % seq_len]*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            x_phase = np.pi*LFM_BW*(x_time**2/(pw_seq[k % seq_len]*1e-6)-x_time)
        # unmod pw_agile freq_agile
        elif (mod_type == 0) & (freq_agile == 1) & (pw_agile == 1) & (pri_agile == 0):
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int)
            x_mag = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_seq[k % seq_len]*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            x_phase = 2*np.pi*freq_seq[k % seq_len]*x_time+freq_off_seq[k % seq_len]
        # LFM pw_agile freq_agile
        elif (mod_type == 1) & (freq_agile == 1) & (pw_agile == 1) & (pri_agile == 0):
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int)
            x_mag = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_seq[k % seq_len]*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            x_phase = np.pi*LFM_BW*(x_time**2/(pw_seq[k % seq_len]*1e-6)-x_time) + 2*np.pi*freq_seq[k % seq_len]*x_time+freq_off_seq[k % seq_len]
        # pri_agile
        elif (mod_type == 0) & (freq_agile == 0) & (pw_agile == 0) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time[j] >= pri_seq[k[j-1]]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_min*1e-6)
            x_phase = phase_off
        # LFM pri_agile
        elif (mod_type == 1) & (freq_agile == 0) & (pw_agile == 0) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time[j] >= pri_seq[k[j-1]]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_min*1e-6)
            x_phase = np.pi*LFM_BW*(x_time_p**2/(pw_min*1e-6)-x_time_p)
        # freq_agile pri_agile
        elif (mod_type == 0) & (freq_agile == 1) & (pw_agile == 0) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time[j] >= pri_seq[k[j-1]]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_min*1e-6)
            x_phase = 2*np.pi*freq_seq[k % seq_len]*x_time_p+freq_off_seq[k % seq_len]
        # LFM freq_agile pri_agile
        elif (mod_type == 1) & (freq_agile == 1) & (pw_agile == 0) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time[j] >= pri_seq[k[j-1]]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_min*1e-6)
            x_phase = np.pi*LFM_BW*(x_time_p**2/(pw_min*1e-6)-x_time_p)+2*np.pi*freq_seq[k % seq_len]*x_time_p+freq_off_seq[k % seq_len]
        # pw_agile pri_agile
        elif (mod_type == 0) & (freq_agile == 0) & (pw_agile == 1) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time[j] >= pri_seq[k[j-1]]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_seq[k % seq_len]*1e-6)
            x_phase = phase_off
        # LFM pw_agile pri_agile
        elif (mod_type == 1) & (freq_agile == 0) & (pw_agile == 1) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time[j] >= pri_seq[k[j-1]]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_seq[k % seq_len]*1e-6)
            x_phase = np.pi*LFM_BW*(x_time_p**2/(pw_seq[k % seq_len]*1e-6)-x_time_p)
        # freq_agile pw_agile pri_agile
        elif (mod_type == 0) & (freq_agile == 1) & (pw_agile == 1) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time[j] >= pri_seq[k[j-1]]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_seq[k % seq_len]*1e-6)
            x_phase = 2*np.pi*freq_seq[k % seq_len]*x_time_p+freq_off_seq[k % seq_len]
        # LFM freq_agile pw_agile pri_agile
        elif (mod_type == 1) & (freq_agile == 1) & (pw_agile == 1) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time[j] >= pri_seq[k[j-1]]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_seq[k % seq_len]*1e-6)
            x_phase = np.pi*LFM_BW*(x_time_p**2/(pw_min*1e-6)-x_time_p)+2*np.pi*freq_seq[k % seq_len]*x_time_p+freq_off_seq[k % seq_len]
        


        x = x_mag*np.exp(1j*x_phase)

        # Add AWGN noise with random power
        SNRdB = random.uniform(SNRdBmin, SNRdBmax)
        SNR = 10**(SNRdB/10)
        sig_pwr = np.mean(np.abs(x)**2)
        sigma2 = sig_pwr/SNR
        noise = np.random.randn(np.size(x))*np.sqrt(sigma2/2) + 1j*np.random.randn(np.size(x))*np.sqrt(sigma2/2)
        x_long = x + noise

        x_short = x_long[np.arange(0, int(np.size(x_long)/D), 1)]
        x_long = x_long[np.arange(0, np.size(x_long), D)]

        xf_short = fft(x_short)
        xf_long  = fft(x_long)


        x_ms = abs(x_short)
        x_ps = np.angle(x_short)
        x_ml = abs(x_long)
        x_pl = np.angle(x_long)
        xf_ms = abs(xf_short)
        xf_ps = np.angle(xf_short)
        xf_ml = abs(xf_long)
        xf_pl = np.angle(xf_long)

        data[i,:,0] = x_ms
        data[i,:,1] = x_ps
        data[i,:,2] = x_ml
        data[i,:,3] = x_pl
        data[i,:,4] = xf_ms
        data[i,:,5] = xf_ps
        data[i,:,6] = xf_ml
        data[i,:,7] = xf_pl

        labels[i,0] = mod_type
        labels[i,1] = freq_agile
        labels[i,2] = pw_agile
        labels[i,3] = pri_agile
        labels[i,4] = pw_min
        labels[i,5] = pw_max
        labels[i,6] = pri_min
        labels[i,7] = pri_max
        labels[i,8] = SNRdB

    return data, labels


In [15]:
def build_model(input_shape,output_size):
    model = Sequential()
    model.add(Conv1D(10, kernel_size=5, padding='same', input_shape=input_shape, activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(20, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(30, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(40, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(50, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(60, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(70, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(80, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(90, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Flatten())
    model.add(Dense(200, activation='relu'))
    model.add(Dense(output_size, activation='relu'))
    return model

In [16]:
model = build_model(input_shape=(1024,8),output_size=2)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_9 (Conv1D)           (None, 1024, 10)          410       
                                                                 
 max_pooling1d_9 (MaxPoolin  (None, 512, 10)           0         
 g1D)                                                            
                                                                 
 conv1d_10 (Conv1D)          (None, 512, 20)           1020      
                                                                 
 max_pooling1d_10 (MaxPooli  (None, 256, 20)           0         
 ng1D)                                                           
                                                                 
 conv1d_11 (Conv1D)          (None, 256, 30)           3030      
                                                                 
 max_pooling1d_11 (MaxPooli  (None, 128, 30)          

In [17]:
# Compile the model
model.compile(loss='MSE', optimizer='Adam', metrics=['accuracy'])
# Setup the callbacks
checkpt = ModelCheckpoint(checkpoint_filepath, save_best_only=True, verbose=0)
tb = TensorBoard(log_dir=log_filepath)
e_stop = EarlyStopping(patience=5)

In [18]:
for i in range(N_train_iter):
    x, y = gen_data()
    
    # Train the model
    hist = model.fit(x, y, epochs=100, validation_split=0.2, verbose=2, batch_size=100,
                    callbacks=[checkpt, tb, e_stop])
    

2024-01-23 12:30:11.135984: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 65536000 exceeds 10% of free system memory.
2024-01-23 12:30:11.272813: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 65536000 exceeds 10% of free system memory.


Epoch 1/100


2024-01-23 12:30:17.582910: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-01-23 12:30:21.328911: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f78ba6bc7d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-23 12:30:21.328975: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro T1000, Compute Capability 7.5
2024-01-23 12:30:21.379801: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-23 12:30:21.550476: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


20/20 - 15s - loss: 32720.1758 - accuracy: 0.9700 - val_loss: 27872.6074 - val_accuracy: 1.0000 - 15s/epoch - 745ms/step
Epoch 2/100


/home/thomas/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


...

2024-01-23 12:30:56.303123: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 65536000 exceeds 10% of free system memory.
2024-01-23 12:30:56.490037: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 65536000 exceeds 10% of free system memory.


20/20 - 1s - loss: 5115.8545 - accuracy: 0.8280 - val_loss: 4053.5173 - val_accuracy: 0.8920 - 769ms/epoch - 38ms/step
Epoch 2/100
20/20 - 1s - loss: 3946.0320 - accuracy: 0.8945 - val_loss: 3707.0134 - val_accuracy: 0.8120 - 543ms/epoch - 27ms/step
Epoch 3/100
20/20 - 1s - loss: 3627.0190 - accuracy: 0.7655 - val_loss: 3596.1953 - val_accuracy: 0.8720 - 541ms/epoch - 27ms/step
Epoch 4/100
20/20 - 1s - loss: 3337.9087 - accuracy: 0.8565 - val_loss: 3173.7427 - val_accuracy: 0.8300 - 530ms/epoch - 27ms/step
Epoch 5/100
20/20 - 1s - loss: 3022.0234 - accuracy: 0.7875 - val_loss: 2845.5166 - val_accuracy: 0.7440 - 525ms/epoch - 26ms/step
Epoch 6/100
20/20 - 1s - loss: 2529.4514 - accuracy: 0.8005 - val_loss: 2726.5601 - val_accuracy: 0.8040 - 530ms/epoch - 26ms/step
Epoch 7/100
20/20 - 0s - loss: 2936.7114 - accuracy: 0.8060 - val_loss: 3184.0015 - val_accuracy: 0.8520 - 449ms/epoch - 22ms/step
Epoch 8/100
20/20 - 0s - loss: 2434.3821 - accuracy: 0.8105 - val_loss: 3081.3550 - val_accurac

2024-01-23 12:31:23.243368: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 65536000 exceeds 10% of free system memory.


20/20 - 1s - loss: 3201.1345 - accuracy: 0.8040 - val_loss: 2362.5330 - val_accuracy: 0.8020 - 849ms/epoch - 42ms/step
Epoch 2/100
20/20 - 1s - loss: 3068.4697 - accuracy: 0.7895 - val_loss: 1987.0498 - val_accuracy: 0.6380 - 512ms/epoch - 26ms/step
Epoch 3/100
20/20 - 0s - loss: 2559.5679 - accuracy: 0.7625 - val_loss: 2025.6017 - val_accuracy: 0.7920 - 409ms/epoch - 20ms/step
Epoch 4/100
20/20 - 0s - loss: 2296.3218 - accuracy: 0.7640 - val_loss: 2412.1802 - val_accuracy: 0.6880 - 389ms/epoch - 19ms/step
Epoch 5/100
20/20 - 1s - loss: 2272.3691 - accuracy: 0.6900 - val_loss: 1695.9010 - val_accuracy: 0.7000 - 502ms/epoch - 25ms/step
Epoch 6/100
20/20 - 0s - loss: 1996.5052 - accuracy: 0.7370 - val_loss: 2035.6277 - val_accuracy: 0.6140 - 389ms/epoch - 19ms/step
Epoch 7/100
20/20 - 0s - loss: 3337.8735 - accuracy: 0.7460 - val_loss: 2398.5239 - val_accuracy: 0.8080 - 407ms/epoch - 20ms/step
Epoch 8/100
20/20 - 0s - loss: 2790.0818 - accuracy: 0.7530 - val_loss: 2617.5024 - val_accurac

KeyboardInterrupt: 